#Simple classification tutorial
Step 1 -  Import our necessary libraries 

In [ ]:
#imports 
import matplotlib.pyplot as plt  #to p lot data
import numpy as np # to do complex math operations
import os # to play with files 
import PIL #to play with images

import tensorflow as tf # we import tensorflow with an alias for ease of use

from tensorflow import keras #human readable API for Deep learning
from tensorflow.keras import layers 
from tensorflow.keras.models import Sequential 

# Step 2 Get our data
we will get a dataset with 300 + iamges of flowers ordered in 5 categories ['daisy','dandaleion','roses','sunflowers', 'tulips']
 

In [ ]:
import pathlib #allows us to create paths for our data

dataset_url = "/content/drive/MyDrive/Playingcards/training_images"
IMAGE_SHAPE = (1056, 691) # (height, width) in no. of pixels
TRAINING_DATA_DIR = str(dataset_url)
datagen_kwargs = dict(rescale=1./255, validation_split=.20)

#data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, untar=True)
#data_dir = pathlib.Path(data_dir)

In [ ]:
batch_size = 32
img_height = 1056
img_width = 691


In [ ]:
#valid_datagen = tf.keras.preprocessing.image_dataset_from_directory(**datagen_kwargs)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  TRAINING_DATA_DIR,
  validation_split= 0.2,
  subset='validation',
  seed=123,
  shuffle=True,
  image_size=IMAGE_SHAPE,
  batch_size= 6
)
#train_datagen = tf.keras.preprocessingimage_dataset_from_directory(**datagen_kwargs)
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  TRAINING_DATA_DIR,
  validation_split= 0.2,
  subset='training',
  seed=123,
  shuffle=True,
  image_size=IMAGE_SHAPE,
  batch_size= 6
  )

In [ ]:
for image_batch, label_batch in train_ds:
  break
image_batch.shape, label_batch.shape

In [ ]:
print(train_ds.class_names)

In [ ]:
class_names= train_ds.class_names
print(class_names)

now that we have our dataset, we can ccheck the dataset and checkl for the amount of images. 

##visualize the data

In [ ]:
plt.figure(figsize= (8,8))

for images,labels in train_ds.take(1):
  for i in range (4):
    ax= plt.subplot(2,2, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

we want to get info from our dataset

Now we need to process our data, to normalize (or to standardize the data). we dont want to deal with RGB values from 0 to 255, so we remap our values to a domain that goes from 0 to 1

In [ ]:
AUTOTUNE= tf.data.AUTOTUNE
print(train_ds)
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds =  val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)

In [ ]:
#a way to use this is...

normalized_ds = train_ds.map(lambda x,y:(normalization_layer(x),y))
image_batch, labels_batch =  next(iter(normalized_ds))
first_image= image_batch[0]

print(np.min(first_image), np.max(first_image))

Another way to normalize the data is to include a layer that does this for you when you create your model. let's do this now!!!!

In [ ]:
num_classes = 4

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape= (img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPool2D(), 
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPool2D(), 
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPool2D(), 
  layers.Flatten(),
  layers.Dense(128, activation= 'relu'),
  layers.Dense(num_classes)
])

#compile the model
we will use adam as optimizer and sparse categorical cross entropy as loss function 

In [ ]:
model.compile(optimizer='adam', 
              loss= tf.keras.losses.SparseCategoricalCrossentropy(from_logits= True),
              metrics=['accuracy'])

In [ ]:
model.summary()

#We are ready to trian our model now

In [ ]:
epochs = 10

history = model.fit(
    train_ds,
    validation_data= val_ds,
    epochs= epochs
)

LEt's visualize the trianing results by creating plots of loss and accuracy on the training and validation sets. 

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss= history.history['loss']
val_loss= history.history['val_loss']


epochs_range =range(epochs)


plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(epochs_range, acc, label='Training accuracy')
plt.plot(epochs_range, val_acc, label='Validation accuracy')
plt.legend(loc= 'lower right')
plt.title('Training and validation accuracy')

plt.subplot(1,2,2)
plt.plot(epochs_range, loss, label='Training loss')
plt.plot(epochs_range, val_loss, label='Validation loss')
plt.legend(loc= 'upper right')
plt.title('Training and validation loss')
plt.show()

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(img_height, 
                                                              img_width,
                                                              3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

In [ ]:
model = Sequential([
  data_augmentation,
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs = 15
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
topredict= "/content/drive/MyDrive/Playingcards/training_images/hearts/AH.jpg"
img = keras.preprocessing.image.load_img(
    topredict, target_size=(img_height, img_width)
)
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)